In [ ]:
# Inference and triangulation pipeline
# 
# Input: 
#       Raw videos from different camera views
# Output:
#       LP output: 2D pose estimation 
#       Anipose output: 3D pose estimation and feature extraction

In [1]:
import os
from glob import glob
import shutil
import pandas as pd
import numpy
import toml

from utils.lp2anipose import lp2anipose_session
from utils.visualization import creat_combined_video
from anipose_yt.compute_angles import process_trial as angles_process
from anipose_yt.triangulate import process_trial as triangulate_process
from anipose_yt.filter_pose import process_trial as filter2d_process
from anipose_yt.label_combined import process_trial as v2d3d_process
from anipose_yt.label_videos_3d import process_trial as v3d_process
from anipose_yt.filter_3d import process_trial as filter3d_process


In [2]:
# Setting
data_dir = r"/media/yiting/NewVolume/Data/Videos"
analysis_dir = r"/media/yiting/NewVolume/Analysis"
session_name = "2025-12-09"
camera_views = ["To", "TL", "TR", "BL", "BR"]

lp_dir = os.path.join(analysis_dir, session_name, "litpose")
ap_dir = os.path.join(analysis_dir, session_name, "anipose")
cal_dir = os.path.join(analysis_dir, "anipose_calibration", "251117_cal")


# Lightning Pose
### Rename and reorganize videos

In [ ]:
os.makedirs(os.path.join(lp_dir, "new_videos"), exist_ok = True)
trials = sorted(os.listdir(os.path.join(data_dir, session_name, "cameras")))
for t in trials:
    trialname_parts = t.split('_')
    for c in camera_views:
        raw_vid_file = os.path.join(data_dir, session_name, "cameras", t, "cam" + c +".mp4")
        new_vid_name = trialname_parts[0] + "_" + trialname_parts[1] + "_" + "cam" + c + ".mp4"
        lp_vid_file = os.path.join(lp_dir, "new_videos", new_vid_name)
        shutil.copyfile(raw_vid_file, lp_vid_file)


### Inference @ Rockfish Cluster

Run inference on new videos  

1. Upload new videos to Rockfish cluster (projects/hand_tracking/lightning-pose-gpu/data/new_videos/session_name)

2. Edit litpose configuration file\
    eval.hydra_paths: path to models to use for prediction\
    eval.test_videos_directory: path to a directory containing videos to run inference on\
    eval.save_vids_after_training: if true, the script will also save a copy of the full video with model predictions overlaid.

3. Edit sbatch script (projects/hand_tracking/lightning-pose-gpu/inference_1gpu.sh)\
###Run the python script\
srun python scripts/predict_new_vids.py --config-path=/vast/doconn15/projects/hand_tracking/lightning-pose-gpu/data --config-name=config_inference.yaml

4. Submit job script on Rockfish cluster\
Terminal:\
cd vast-doconn15/projects/hand_tracking/lightning-pose-gpu\
sbatch inference_1gpu.sh


# Anipose

### Setting
Create a configuration file

In [3]:
config_file = os.path.join(ap_dir, "anipose_config.toml")
# Load config file
config = toml.load(config_file)


### Convert and reorganize LP outputs
1. Download Lightning Pose outputs from Rockfish cluster (outputs/YYYY-MM-DD/HH-MM-SS/video_preds)
2. Convert Lightning pose 2d outputs (.csv) to Anipose inputs (.hdf)

In [ ]:
lp_2d_dir = os.path.join(lp_dir, "video_preds")
ap_2d_dir = os.path.join(ap_dir, "pose_2d")
os.makedirs(ap_2d_dir, exist_ok = True)
lp2anipose_session(lp_2d_dir, ap_2d_dir, camera_views)

### Filtering 2D data
The filter applied over the 2D data functions as a threshold filter. Predicted labels that do not fall within the threshold are removed and replaced with a new prediction that is determined by interpolating. In config.toml, the parameter spline can be set to true for interpolation using a cubic spline, or false for linear interpolation.

In [ ]:
ap_2d_dir = os.path.join(ap_dir, "pose_2d")
trials = sorted(os.listdir(ap_2d_dir))
for t in trials:
    filter2d_process(config, session_name, t)

### Triangulation

In [ ]:
ap_2d_dir = os.path.join(ap_dir, "pose_2d")
trials = sorted(os.listdir(ap_2d_dir))
for t in trials:
    triangulate_process(config, session_name, t)

### Filtering 3D data
The filter applied over the 3D data functions as a threshold filter. Predicted labels that do not fall within the threshold are removed and replaced with a new prediction that is determined by interpolating. 

In [ ]:
if config['filter']['enabled']:
    ap_2d_dir = os.path.join(ap_dir, "pose_2d_filter")
trials = sorted(os.listdir(ap_2d_dir))
for t in trials:
    filter3d_process(config, session_name, t)

### Feature Extraction
Compute hand configuration parameters (length, angle, etc)

In [4]:
if config['filter']['enabled']:
    ap_2d_dir = os.path.join(ap_dir, "pose_2d_filter")
trials = sorted(os.listdir(ap_2d_dir))
for t in trials:
    angles_process(config, session_name, t)

/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-00-13_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-00-26_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-00-40_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-00-53_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-01-07_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-01-20_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-01-34_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-01-48_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-02-01_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-02-14_angles.csv
/media/yiting/NewVolume/Analysis/2025-12-09/anipose/angles/2025-12-09_09-02-27_angles.csv
/media/yit

### Visualization

In [ ]:
# Create labeled 3d videos
ap_2d_dir = os.path.join(ap_dir, "pose_2d")
trials = sorted(os.listdir(ap_2d_dir))
for t in trials[4:8]:
    v3d_process(config, session_name, t, filtered=True)

In [ ]:
# Create combined videos that have reprojected labeled 2d videos, labeled 3d videos, and angle traces across time. 
ap_2d_dir = os.path.join(ap_dir, "pose_2d")
trials = os.listdir(ap_2d_dir)
for t in trials[4:8]:
    v2d3d_process(config, session_name, t)

In [ ]:
# Create combined videos that have a 2d video from a single camera view and selected angles
ap_2d_dir = os.path.join(ap_dir, "pose_2d")
trials = sorted(os.listdir(ap_2d_dir))
camera_view = 'camTL'
feature_columns = ['index_mcp', 'middle_mcp', 'ring_mcp']
# feature_columns = ['index_dip', 'index_pip', 'index_mcp']
# feature_columns = ['index_pip', 'middle_pip', 'ring_pip', 'ring_pip']
# feature_columns = ['index_dip', 'middle_dip', 'ring_dip', 'ring_dip']
# feature_columns = ['middle_dip', 'middle_pip', 'middle_mcp']
# feature_columns = ['thumb_ip', 'thumb_mcp']

if len(trials) > 0:
    os.makedirs(os.path.join(analysis_dir, session_name, 'anipose', 'videos_v2d_angles'), exist_ok=True)
for t in trials[0:3]:
    video_path = r'/home/yiting/Downloads/2024-11-22_10-10-24_2d3d.mp4'
    # video_path = os.path.join(data_dir, session_name, 'cameras', t, t + '_' + camera_view + '.mp4')
    traces_csv = os.path.join(analysis_dir, session_name, 'anipose', 'angles', t + '_angles.csv')
    output_path = os.path.join(analysis_dir, session_name, 'anipose', 'videos_v2d_angles', t + '_' + camera_view + '_mcp.mp4')
    if os.path.exists(output_path):
        continue
    creat_combined_video(video_path, traces_csv, output_path, feature_columns)